# ASCAT SM 数据预处理

## 导入库

In [4]:
import netCDF4
import numpy as np
from pyresample import AreaDefinition, kd_tree, SwathDefinition

## 读数据

In [5]:
sm_file = r"C:\Datasets\RS\Metop-ASCAT-SM-12.5KM\2018\W_XX-EUMETSAT-Darmstadt,SURFACE+SATELLITE,METOPB+ASCAT_C_EUMP_20180801013900_30451_eps_o_125_ssm_l2.nc"

sm_obj = netCDF4.Dataset(sm_file)
# 关闭netCDF4对数据的自动掩膜和缩放
sm_obj.set_auto_mask(False)
sm_obj.set_auto_scale(False)

lon = sm_obj.variables["lon"][:] * 1e-6
lat = sm_obj.variables["lat"][:] * 1e-6
wetland_f = sm_obj.variables["wetland_flag"][:]
sm_arr = sm_obj.variables["soil_moisture"][:] * 0.01
topography_f = sm_obj.variables["topography_flag"][:]
sm_error = sm_obj.variables["soil_moisture_error"][:] * 0.01

sm_obj.close()

## 数据掩膜

1. 掩膜掉`wetland_flag`大于15%的
2. 掩膜掉`topography_flag`大于20的
3. 掩膜掉`sm`无效值，乘以0.01后是655.35，直接用大于600判断了
4. 掩膜掉`sm_err`大于10%的。`sm`是无效值的地方`sm_err`也是无效值

In [6]:
mask_conditions = (sm_error > 10) | (wetland_f > 15) | (topography_f > 20) | (sm_arr > 600)
sm_arr_masked = np.where(mask_conditions, np.nan, sm_arr)

## 数据重投影

重投影到0.125°间隔的等经纬度网格

In [7]:
swath = SwathDefinition(lon, lat)
grid = AreaDefinition.from_epsg(4326, 0.125)
sm_arr_grid = kd_tree.resample_nearest(swath, sm_arr_masked, grid, radius_of_influence=12500, fill_value=np.nan)

## 输出到新的nc文件

写NC文件的套路

1. 新建维度
2. 新建变量
3. 写入变量值

In [8]:
grid_lon, grid_lat = grid.get_lonlats()

output_obj = netCDF4.Dataset("new.nc", "w")

output_obj.createDimension("lon", grid_lon.shape[1])
output_obj.createDimension("lat", grid_lat.shape[0])

output_lon = output_obj.createVariable("lon", "f4", ("lon",))
output_lat = output_obj.createVariable("lat", "f4", ("lat",))
output_sm = output_obj.createVariable("sm", "f4", ("lat","lon",), fill_value=np.nan)

output_sm[:] = sm_arr_grid
output_lon[:] = grid_lon[0, :]
output_lat[:] = grid_lat[:, 0]

output_obj.close()